In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objects as go

df = pd.read_csv('/kaggle/input/ediblefoods-1961-to-2011/EdibleFoods-1961-2011.csv')
df.head()

In [ ]:
p = df.groupby(['Year','Country_abr','Country','Potatoes'])['Wheat'].sum().reset_index()

p = p.drop('Wheat',axis=1)

for i in range(len(p['Potatoes'])):
    if p['Potatoes'][i] == '*':
        p.loc[i, "Potatoes"] = float('NaN')
        
p = p.dropna()
p

In [ ]:
p['Country'].unique()
# Sorts the country values, which is necessary for the visualization*****
p = p.sort_values('Country')

In [ ]:
'''
    Maybe we can use a line chart idiom to show how the importance of 
    potatoes to various (user chosen) countries changes.
    
    Dropdowns:
    https://plotly.com/python/dropdowns/
'''

In [ ]:
p.Potatoes = pd.to_numeric(p.Potatoes)

In [ ]:
fd = []
for x in p['Country'].unique():
    fd.append(p.groupby(['Country','Year'])['Potatoes'].sum()[x].reset_index())
fd[0]

In [ ]:
homeDf = p.groupby(['Country','Year'])['Potatoes'].sum().reset_index()
homeDf = homeDf.groupby(['Year'])['Potatoes'].mean().reset_index()
homeDf

In [ ]:

fig = go.Figure(
    go.Scatter(x=fd[0].Year,
               y=fd[0].Potatoes,
               name="Low Average",
               visible=True,
               line=dict(color="#F06A6A", dash="solid"))
)
fig.update_layout(
    #width=800,
    #height=900,
    #autosize=False,
    title="low",
    margin=dict(t=0, b=0, l=0, r=0),
    template="plotly_white",
)
fig.show()

# https://www.youtube.com/watch?v=uzosQuETMKo
# https://stackoverflow.com/questions/65710352/how-can-i-add-a-button-or-dropdown-in-a-plot-created-using-plotly-in-python

In [ ]:
fig = go.Figure()

L = []

fig.update_layout(
    #width=800,
    #height=900,
    #autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    template="plotly_white",
)

for i in range(p['Country'].nunique()):
    L.append(False)
    fig.add_trace(
        go.Scatter(x=fd[i].Year,
               y=fd[i].Potatoes,
               name="Low Average",
               visible=False,
               line=dict(color="#F06A6A", dash="dash")))


fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{"visible":L}],
                    label="3D Surface",
                    method="update"
                ),
                dict(
                    args=[{"visible":L}],
                    label="Heatmap",
                    method="update"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

fig.update_layout(
    annotations=[
        dict(text="Location", x=0, xref="paper", y=1.06, yref="paper",
                             align="left", showarrow=False),
    ])


fig.show()

In [ ]:
# Source: https://github.com/lforeman2/plotlyDropDownMenu/blob/main/plotlyDemo_groupedHistogram.ipynb


traces = []
buttons = []
title = "Mean Potato Supplies per Year"
m=0
for i in range(len(fd)):
    visible = [False] * (len(fd)+1)
    
    traces.append(
        go.Scatter( 
            x=fd[i].Year,
            y=fd[i].Potatoes,
            visible=False,
            name=p.Country.unique()[i],
            line=dict(color="#F06A6A", dash="solid")
        )
    )
    if i == 0:
        # maybe add an average overall country thingy
        visible[len(fd)] = True
        buttons.append(dict(label="Home",
                        method="update",
                        args=[{"visible":visible},
                              {"title":f"{(title)}"}]))
        visible[len(fd)] = False
    visible[m] = True
    m+=1
    
    buttons.append(dict(label=p.Country.unique()[i],
                        method="update",
                        args=[{"visible":visible},
                              {"title":f"{(p.Country.unique()[i])}"}]))

updatemenus = [{'active':0, "buttons":buttons}]

# Home, average of all countries
traces.append(
        go.Scatter( 
            x=homeDf.Year,
            y=homeDf.Potatoes,
            visible=True,
            name="Global Mean",
            line=dict(color="#F06A6A", dash="solid")
        )
    )

#print(traces)

fig = go.Figure(data=traces,
                 layout=dict(updatemenus=updatemenus))
fig.update_layout(title=title, barmode='overlay',template="plotly_white",)
fig.update_xaxes(title=str("Year"))
fig.update_yaxes(title=str("Supplies (g/person/day)"))
fig.show()
fig.write_html("demowidget.html")